In [2]:
import SAP_GUI
import win32com.client
import pandas as pd
import subprocess
from time import sleep
from datetime import datetime
from datetime import timedelta
from pywinauto import Desktop
from pywinauto.application import Application
import pyautogui
import numpy as np
 
#loading=Search_CO60(session,variant,"98889984")

Ordenes=[]  #Lista para traer las ordenes como index
InfoPES=[]     #Lista Para traer la descripcion PES
InfoMAN=[]    #Lista Para traer la descripcion MAN
dic={}       #Lista Para traer la descripcion MAN

"""------------------------------------------FUNCION PARA HACER FIRMAS----------------------------------------------"""
def Firmar(session,orden,Area,Info):
    # table.selectedRows= j
    session.findById("wnd[0]/tbar[1]/btn[5]").press()
    session.findById("wnd[0]/usr/shellcont/shell")
    width, height = pyautogui.size()
    # Calcular las coordenadas del centro de la pantalla
    (center_x,center_y) = width // 2 , height // 2
    # Mover el puntero del mouse a las coordenadas del centro
    pyautogui.moveTo(center_x, center_y)
    pyautogui.click()
    try:
        app = Application().connect(title="Process Manufacturing Cockpit VVALENCIAO")  #Puede cambiar la ventana
        dlg = app.top_window()
        dlg.child_window(title="Control  Container", class_name="Shell Window Class").click_input()
        pyautogui.press('tab')
        pyautogui.press('enter')
        # Obtener las dimensiones de la pantalla
        screen_width, screen_height = pyautogui.size()
        # Mover el cursor al centro de la pantalla
        pyautogui.moveTo(screen_width/2, screen_height/2)
        # Desplazarse hacia abajo utilizando la función scroll
        pyautogui.scroll(-30600)
        sleep(1)
        dlg.child_window(title="Control  Container", class_name="Shell Window Class").click_input()
        pyautogui.press('tab')
        pyautogui.press('tab')
        try:
            pyautogui.write("VVALENCIAO")
            pyautogui.press('enter')
            session.findById("wnd[1]/usr/pwdSIGN_POPUP_STRUC-PASSWORD").text= password
            session.findById("wnd[1]/tbar[0]/btn[0]").press()  #Boton Check para cerrar orden 
            #session.findById("wnd[1]/tbar[0]/btn[12]").press()  #Boton Cancelar
            
            # Aqui se manda a lista de pesados

            Info.append("Se firmo "+Area)   #Colocar Dispensado o Fabricación
            session.findById("wnd[0]/tbar[0]/btn[15]").press()
            print("Se firmo la orden FAB"+" "+str(orden))
            return(Info)
        except:
            # Ordenes.append(orden)                     Coloco la orden por si no se ha firmado
            Info.append("No se firmo "+Area)  #Colocar Dispensado o Fabricación
            session.findById("wnd[0]/tbar[0]/btn[15]").press()
            return(Info)
    except:
        print("No encontro La orden"+" "+str(orden))  
        Info.append("No encontro orden")    #Revisar este except se puede omitir 

"""------------------------------------------FUNCION PARA BUSCAR EN C060----------------------------------------------"""

#Search_CO60(session,variant,)
def Search_CO60(session,variant,orden=None):
    try:    
        session.StartTransaction("CO60") 
        session.findById("wnd[0]").maximize()
        session.findById("wnd[0]/tbar[1]/btn[17]").press()
        variants =session.findById("wnd[1]/usr/cntlALV_CONTAINER_1/shellcont/shell")  #Contenedor de variantes
        row=variants.RowCount  #Count total rows
        lis=[variants.GetCellValue(i,"VARIANT") for i in range(0,row)] 
        indice=[indice for indice, dato in enumerate(lis) if dato == variant]
        variants.selectedRows = indice[0]
        session.findById("wnd[1]/tbar[0]/btn[2]").press()
        if orden==None:
            None
        else:
            try:
                session.findById("wnd[0]/usr/ctxtS_AUFNR-LOW").text = orden
                session.findById("wnd[0]/tbar[1]/btn[8]").press()
            except:
                None
        # Comparar las firmas y realizar
        comparative=["DISPENSADO","FABRICACION"]
        table=session.findById("wnd[0]/usr/cntlCONTAINER/shellcont/shell")
        Len=table.RowCount
        if Len==2:
            Ordenes.append(orden)
            try:                             #Firma las 2 filas de la tabla
                for j in range(0,Len):
                    table.selectedRows= j
                    if j==0:
                        Firmar(session,orden,comparative[j],InfoPES)
                    else:
                        Firmar(session,orden,comparative[j],InfoMAN)
            except:                          #En caso de error, se firmo primera fila se quito y cambio la longuitud
                table.selectedRows= 0
                Firmar(session,orden,comparative[1],InfoMAN)
        elif Len==1:
            Ordenes.append(orden)
            column=table.GetCellValue(0,"ADRES")
            if comparative[0] in column:
                Firmar(session,orden,comparative[0],InfoPES)
                InfoMAN.append("No encontro orden "+comparative[1])
            else:
                Firmar(session,orden,comparative[1],InfoMAN)
                InfoPES.append("No encontro orden "+comparative[0])
        else:
            dic[orden]="No se encontro orden"
            print("No encontro la orden")
    except:
        dic[orden]="No se encontro orden"

user="vvalenciao"
password="Abril2023%"
Iniciar=SAP_GUI.SessionSAP(user,password)
Iniciar.findById("wnd[0]").resizeWorkingPane(188,25,False)
session=Iniciar

#Search C060
Transsaccion="CO60"
variant="MNF FABRICACIO"

#Search Cooispi
provisionCooispi="Z_ZO_KANBAN"
variantCooispi="jgiraldoo"
TranssaccionCooispi="cooispi"
disposicion="/KANBANE50"


dateIni=(datetime.now()-timedelta(days=2)).strftime("%d-%m-%Y").replace("-",".")
dateFin=(datetime.now()-timedelta(days=0)).strftime("%d-%m-%Y").replace("-",".")
#dateFin=datetime.now().strftime("%d-%m-%Y").replace("-",".")


SAP_GUI.Search_COOISPI(TranssaccionCooispi,session,variantCooispi,provisionCooispi,disposicion,dateIni,dateFin)

#Export Cooispi
Name="ManufacturaE50"
SAP_GUI.Export_TXT2(Name,session)

#Load Cooispi
Ruta="C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Documentos 1\\SAP\\SAP GUI\\"+str(Name)+".txt"
data = pd.read_csv(Ruta, skiprows=1, delimiter='\t') #Mb52 skip=1

#Load Cooispi len=3
test=data["Orden"]
#Read ordenes
for i in test:
    Search_CO60(session,variant,i)

SAP_GUI.Close_session(session)


c:\Users\prac.ingindustrial2\Anaconda3\lib\site-packages\pywinauto\application.py:1085: UserWarning: 32-bit application should be automated using 32-bit Python (you use 64-bit Python)
  warnings.warn(


Se firmo la orden FAB 89529109


c:\Users\prac.ingindustrial2\Anaconda3\lib\site-packages\pywinauto\application.py:1085: UserWarning: 32-bit application should be automated using 32-bit Python (you use 64-bit Python)
  warnings.warn(


Se firmo la orden FAB 89529109


c:\Users\prac.ingindustrial2\Anaconda3\lib\site-packages\pywinauto\application.py:1085: UserWarning: 32-bit application should be automated using 32-bit Python (you use 64-bit Python)
  warnings.warn(


...

Se firmo la orden FAB 98891658


c:\Users\prac.ingindustrial2\Anaconda3\lib\site-packages\pywinauto\application.py:1085: UserWarning: 32-bit application should be automated using 32-bit Python (you use 64-bit Python)
  warnings.warn(


Se firmo la orden FAB 98891659


c:\Users\prac.ingindustrial2\Anaconda3\lib\site-packages\pywinauto\application.py:1085: UserWarning: 32-bit application should be automated using 32-bit Python (you use 64-bit Python)
  warnings.warn(


Se firmo la orden FAB 98891659


In [11]:
#Load Cooispi
Ruta="C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Documentos 1\\SAP\\SAP GUI\\"+str(Name)+".txt"
Dat = pd.read_csv(Ruta, skiprows=1, delimiter='\t') #Mb52 skip=1
len(Dat["Orden"])

if (len(InfoMAN)==len(InfoPES)) and (len(Ordenes)==len(InfoMAN)):
    data=pd.DataFrame(zip(InfoPES[0:len(Ordenes)+1],InfoMAN[0:len(Ordenes)+1]),index=Ordenes[0:len(Ordenes)+1],columns=["DISPENSADO","FABRICACION"])
else:
    None
data


,DISPENSADO,FABRICACION
89529109,Se firmo DISPENSADO,Se firmo FABRICACION
89529254,Se firmo DISPENSADO,Se firmo FABRICACION
89529255,Se firmo DISPENSADO,Se firmo FABRICACION
41066886,No se firmo DISPENSADO,Se firmo FABRICACION
41066956,Se firmo DISPENSADO,Se firmo FABRICACION
...,...,...
98891579,Se firmo DISPENSADO,Se firmo FABRICACION
98891647,No se firmo DISPENSADO,No se firmo FABRICACION
98891654,Se firmo DISPENSADO,Se firmo FABRICACION
98891658,Se firmo DISPENSADO,Se firmo FABRICACION


In [12]:
try:
    dataNo=pd.DataFrame(dic.values(),index=dic.keys())
except:
    dataNo=pd.DataFrame()

dataNo


""


In [13]:
from datetime import datetime
from datetime import timedelta

"""Send email"""

correos="prac.ingindustrial2@prebel.com.co;vanessa.valencia@prebel.com.co"
now=datetime.now()
def send_emails(*args,emails="",htmlbody="",subject=""):
    email=emails
    outlook=win32com.client.Dispatch("outlook.application")
    mail=outlook.CreateItem(0)
    mail.Subject=subject+" "+datetime.now().strftime('%#d %b %Y %H:%M')
    mail.To=email
    mail.HTMLBody=htmlbody.format(*args)
    mail.Send()

def style_df(df):
    return df.style \
        .set_table_styles([{'selector': "table,tr,th,td", 'props': [("border", "1px solid"), ('color', '#000'),("text-align","center")]}]) \
        .highlight_between(subset=["DISPENSADO"],color='#FF5733',left="No se firmo DISPENSADO",right="No se firmo DISPENSADO") \
        .highlight_between(subset=["FABRICACION"],color='#FF5733',left="No se firmo FABRICACION",right="No se firmo FABRICACION") \

def style_df_stand(df):
    return df.style \
        .set_table_styles([{'selector': "table,tr,th,td", 'props': [("border", "1px solid"), ('color', '#000'),("text-align","center")]}]) \



html="""
    <h2 style="text-align: center"> REPORTE FIRMAS DISPENSADO y FABRICACIÓN</h2>
    <p> Por medio del presente informe se evidencia las firmas tanto de Dispensado y Fabricación que se llevado a cabo en el BATCH RECORD
        electronicó.</p>

    <h4 style="color: red;" > Advertencia: Las firmas resaltadas en rojo son las que no se han firmado.</h4>
    
    <div"> {0} </div>

    <h4 style="color: black;" > Nota: A continuación se evidencian las ordenes que no se han encontrado, por lo cual se recomienda revisarlas,
        en tal caso no se evidencie la tabla hacer caso omiso a esta nota.</h4>

    <div"> {1} </div>
    <p> Anticipo sinceros agradecimientos. </p>
 """

Send = style_df(data)  #Style between LI and LS     #Data de firmas
Send2 = style_df_stand(dataNo)

send_emails(Send.to_html(),Send2.to_html(),emails=correos,htmlbody=html,subject="REPORTE FIRMAS DISPENSADO y FABRICACIÓN")


In [6]:
import SAP_GUI
import win32com.client
import pandas as pd
import subprocess
from time import sleep
from datetime import datetime
from datetime import timedelta
from pywinauto import Desktop
from pywinauto.application import Application
import pyautogui
import numpy as np




user="vvalenciao"
password="Abril2023%"
Iniciar=SAP_GUI.SessionSAP(user,password)
Iniciar.findById("wnd[0]").resizeWorkingPane(188,25,False)
session=Iniciar

#Search C060
Transsaccion="CO60"
variant="MNF FABRICACIO"

#Search Cooispi
provisionCooispi="Z_ZO_KANBAN"
variantCooispi="jgiraldoo"
TranssaccionCooispi="cooispi"
disposicion="/KANBANE50"


dateIni=(datetime.now()-timedelta(days=1)).strftime("%d-%m-%Y").replace("-",".")
dateFin=(datetime.now()-timedelta(days=0)).strftime("%d-%m-%Y").replace("-",".")
#dateFin=datetime.now().strftime("%d-%m-%Y").replace("-",".")


SAP_GUI.Search_COOISPI(TranssaccionCooispi,session,variantCooispi,provisionCooispi,disposicion,dateIni,dateFin)

<COMObject <unknown>>

In [7]:
#Export Cooispi
Name="ManufacturaE50"
SAP_GUI.Export_TXT2(Name,session)
#Load Cooispi



In [8]:
if (len(InfoMAN)==len(InfoPES)) and (len(Ordenes)==len(InfoMAN)):
    data=pd.DataFrame(zip(InfoPES[0:len(Ordenes)+1],InfoMAN[0:len(Ordenes)+1]),index=Ordenes[0:len(Ordenes)+1],columns=["DISPENSADO","FABRICACION"])
else:
    None
data

,DISPENSADO,FABRICACION
89529109,Se firmo DISPENSADO,Se firmo FABRICACION
89529254,Se firmo DISPENSADO,Se firmo FABRICACION
89529255,Se firmo DISPENSADO,Se firmo FABRICACION
41066886,No se firmo DISPENSADO,Se firmo FABRICACION
41066956,Se firmo DISPENSADO,Se firmo FABRICACION
...,...,...
98891579,Se firmo DISPENSADO,Se firmo FABRICACION
98891647,No se firmo DISPENSADO,No se firmo FABRICACION
98891654,Se firmo DISPENSADO,Se firmo FABRICACION
98891658,Se firmo DISPENSADO,Se firmo FABRICACION


In [9]:
user="vvalenciao"
password="Abril2023%"
Iniciar=SAP_GUI.SessionSAP(user,password)



In [10]:
#Search Cooispi
provisionCooispi="Z_ZO_KANBAN"
variantCooispi="jgiraldoo"
TranssaccionCooispi="cooispi"
disposicion="/KANBANE50"


session=Iniciar
SAP_GUI.Search_COOISPI(TranssaccionCooispi,session,variantCooispi,provisionCooispi,disposicion,dateIni,dateFin)

#Export Cooispi
Name="ManufacturaE50"
SAP_GUI.Export_TXT2(Name,session)

#Load Cooispi
Ruta="C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Documentos 1\\SAP\\SAP GUI\\"+str(Name)+".txt"
data = pd.read_csv(Ruta, skiprows=1, delimiter='\t') #Mb52 skip=1